# 02_NBS_ Arima Model (Time-Series) Forecasting

## Purpose

The purpose of this notebook is to build ARIMA model and deep diving about ARIMA model.

## Methodology
The approach used to made this experiment are: find the orders parameters (p, q, d), build ARIMA model, use AUTO ARIMA model, plot the projection of forecasting result.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- Find the order parameters (p, q, d);
- Build ARIMA model;
- Use AUTO ARIMA Model.

## Results
To do.

## Suggested next steps
To do.

# Setup 
## Library Import

In [1]:
import numpy as np
import pandas as pd


## Local Library Import

In [2]:
import os

## Parameter Definition

In [3]:
DATA_PATH = "../data/second_dataset.csv"

# Data Import

In [4]:
df = pd.read_csv(DATA_PATH)
df.head()

,date,value
0,1991-07-01,3.526591
1,1991-08-01,3.180891
2,1991-09-01,3.252221
3,1991-10-01,3.611003
4,1991-11-01,3.565869


In [5]:
df.shape

(204, 2)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    204 non-null    object 
 1   value   204 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.3+ KB
